# 数据操作
:label:`sec_ndarray`

为了完成任何事情，
我们需要某种方式来存储和处理数据。
通常，我们需要对数据做两件重要的事情：
（i）获取数据；
（ii）在数据进入计算机后对其进行处理。
如果没有办法存储数据，获取数据是没有意义的，
因此，让我们从$n$-维数组开始动手，
我们也称之为*张量*。
如果你已经熟悉了NumPy科学计算包，
这将非常简单。
对于所有现代深度学习框架，
*张量类*（MXNet中的`ndarray`，
PyTorch和TensorFlow中的`Tensor`）
类似于NumPy的`ndarray`，
并增加了一些关键特性。
首先，张量类支持自动微分。
其次，它利用GPU加速数值计算，
而NumPy只能在CPU上运行。
这些特性使得神经网络
既易于编码又运行快速。



## 开始


请提供翻译结果，无需任何额外解释，并移除。如果翻译是不必要的（例如专有名词、代码等），返回原文。无解释。无注释。确保输出格式与输入格式完全相同。

(**首先，我们导入PyTorch库。
请注意，包名是`torch`。**)

In [228]:
import torch

[**张量表示数值的（可能是多维的）数组。**]
在一维情况下，即数据只需要一个轴时，
张量被称为*向量*。
在有两个轴的情况下，张量被称为*矩阵*。
当有 $k > 2$ 个轴时，我们不再使用专门的名称，
而只是将该对象称为 $k^\textrm{th}$-*阶张量*。

PyTorch 提供了多种函数来创建预填充值的新张量。例如，通过调用 `arange(n)`，我们可以创建一个均匀分布的值的向量，从 0（包括）开始到 `n`（不包括）。默认情况下，间隔大小为 $1$。除非另有说明，新张量存储在主内存中，并指定用于基于 CPU 的计算。

In [229]:
x = torch.arange(12, dtype=torch.float32)
x

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

这些值中的每一个都被称为张量的*元素*。
张量`x`包含12个元素。
我们可以通过张量的`numel`方法检查张量中元素的总数。

In [230]:
x.numel()

12

（**我们可以访问张量的*形状***）
（每个轴的长度）
通过检查其`shape`属性。
因为这里处理的是一个向量，
`shape`只包含一个元素
并且与大小相同。

In [231]:
x.shape

torch.Size([12])

我们可以通过调用`reshape`来[**改变张量的形状而不改变其大小或值**]。例如，我们可以将形状为(12,)的向量`x`转换为形状为(3, 4)的矩阵`X`。这个新的张量保留了所有元素，但将它们重新配置成一个矩阵。请注意，我们的向量中的元素是按行依次排列的，因此`x[3] == X[0, 3]`。

In [232]:
X = x.reshape(3, 4)
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

请注意，为`reshape`指定每个形状组件是多余的。因为我们已经知道张量的大小，所以可以根据其余部分推断出形状的一个组件。例如，给定一个大小为$n$的张量和目标形状（$h$，$w$），我们知道$w = n/h$。为了自动推断形状的一个组件，我们可以将应该自动推断的形状组件设置为`-1`。在我们的情况下，除了调用`x.reshape(3, 4)`之外，还可以等效地调用`x.reshape(-1, 4)`或`x.reshape(3, -1)`。

实践者经常需要处理初始化为全0或全1的张量。[**我们可以通过`zeros`函数构造一个所有元素都设置为0**] (~~或1~~)且形状为(2, 3, 4)的张量。

In [233]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

同样，我们可以通过调用`ones`来创建一个全为1的张量。

In [234]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

我们经常希望从给定的概率分布中随机（且独立地）抽取每个元素。例如，神经网络的参数通常会被随机初始化。以下代码片段创建了一个张量，其元素是从均值为0、标准差为1的标准高斯（正态）分布中抽取的。

In [235]:
torch.randn(3, 4)

tensor([[ 0.3620,  0.0494,  0.5405, -0.3573],
        [-0.0776, -0.9660, -0.2192,  1.0989],
        [-0.4543, -0.2755,  1.5128, -2.6399]])

最后，我们可以通过
[**为每个元素提供确切的值**]
通过提供（可能是嵌套的）包含数值字面量的Python列表来构建张量。
这里，我们使用一个列表的列表来构造一个矩阵，
其中最外层的列表对应于轴0，
内层列表对应于轴1。

In [236]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

## 索引和切片

与 Python 列表一样，
我们可以通过索引（从0开始）访问张量元素。
要根据列表末尾的位置访问元素，
我们可以使用负索引。
最后，我们可以通过切片（例如，`X[start:stop]`）访问整个范围的索引，
返回的值包括第一个索引（`start`）*但不包括最后一个*（`stop`）。
最后，当只为一个 $k^\textrm{th}$ 阶张量指定一个索引（或切片）时，
它将应用于轴0。
因此，在以下代码中，
[**`[-1]` 选择最后一行，而 `[1:3]` 选择第二和第三行**]。

In [237]:
X[-1], X[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

除了阅读它们，（**我们还可以通过指定索引来*写入*矩阵的元素。**）

In [238]:
X[1, 2] = 17
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5., 17.,  7.],
        [ 8.,  9., 10., 11.]])

如果我们要[**为多个元素分配相同的值，
我们在赋值操作的左侧应用索引。**]
例如，`[:2, :]` 访问
第一和第二行，
其中 `:` 沿轴1（列）取所有元素。
虽然我们讨论了矩阵的索引，
但这同样适用于向量
以及超过二维的张量。

In [239]:
X[:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

## 操作

现在我们已经知道如何构造张量
以及如何读取和写入它们的元素，
我们可以开始使用各种数学操作来处理它们。
其中最有用的是*逐元素*操作。
这些操作将标准的标量操作
应用于张量的每个元素。
对于接受两个张量作为输入的函数，
逐元素操作会对每对对应的元素应用某种标准的二元运算符。
我们可以从任何将标量映射到标量的函数
创建一个逐元素函数。

在数学符号中，我们将这样的
*一元*标量运算符（接受一个输入）
表示为签名
$f: \mathbb{R} \rightarrow \mathbb{R}$。
这仅仅意味着该函数将
任何实数映射到另一个实数。
大多数标准运算符，包括像 $e^x$ 这样的单目运算符，都可以逐元素应用。

In [240]:
torch.exp(x)

tensor([162754.7969, 162754.7969, 162754.7969, 162754.7969, 162754.7969,
        162754.7969, 162754.7969, 162754.7969,   2980.9580,   8103.0840,
         22026.4648,  59874.1406])

同样，我们表示*二元*标量运算符，
这些运算符将实数对
映射到一个（单一的）实数
通过签名
$f: \mathbb{R}, \mathbb{R} \rightarrow \mathbb{R}$。
给定任意两个*形状相同*的向量$\mathbf{u}$
和$\mathbf{v}$，以及一个二元运算符$f$，我们可以通过设置$c_i \gets f(u_i, v_i)$对于所有的$i$来生成一个向量$\mathbf{c} = F(\mathbf{u},\mathbf{v})$，
其中$c_i, u_i$和$v_i$分别是向量$\mathbf{c}, \mathbf{u}$和$\mathbf{v}$的第$i$个元素。
这里，我们通过*提升*标量函数
为逐元素向量操作产生了向量值的
$F: \mathbb{R}^d, \mathbb{R}^d \rightarrow \mathbb{R}^d$。
常见的标准算术运算符
加法 (`+`)、减法 (`-`)、
乘法 (`*`)、除法 (`/`)、
和幂运算 (`**`)
都已经被*提升*为任意形状的同形张量的逐元素操作。

In [241]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

除了逐元素计算之外，我们还可以执行线性代数运算，例如点积和矩阵乘法。我们将在:numref:`sec_linear-algebra`中详细讨论这些内容。

我们还可以将多个张量**连接**起来，将它们端对端堆叠以形成一个更大的张量。我们只需要提供一个张量列表，并告诉系统沿着哪个轴进行连接。下面的例子展示了当我们沿着行（轴0）而不是列（轴1）连接两个矩阵时会发生什么。我们可以看到，第一个输出的轴0长度（$6$）是两个输入张量的轴0长度之和（$3 + 3$）；而第二个输出的轴1长度（$8$）是两个输入张量的轴1长度之和（$4 + 4$）。

In [242]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

有时，我们想要
[**通过*逻辑语句*构建一个二进制张量。**]
以 `X == Y` 为例。
对于每个位置 `i, j`，如果 `X[i, j]` 和 `Y[i, j]` 相等，
则结果中的相应条目取值 `1`，
否则取值 `0`。

In [243]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

[**对张量中的所有元素求和**]会得到一个只有一个元素的张量。

In [244]:
X.sum()

tensor(66.)

## 广播
:label:`subsec_broadcasting`

到目前为止，你已经知道如何对两个形状相同的张量执行逐元素二元操作。在某些条件下，即使形状不同，我们仍然可以通过调用*广播机制*来执行逐元素二元操作。广播的工作原理如下两步程序：(i) 通过沿长度为1的轴复制元素来扩展一个或两个数组，使得经过这种变换后，两个张量具有相同的形状；(ii) 对生成的数组执行逐元素操作。

In [245]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

由于`a`和`b`分别是$3\times1$和$1\times2$矩阵，它们的形状不匹配。广播机制通过沿列复制矩阵`a`和沿行复制矩阵`b`来生成一个更大的$3\times2$矩阵，然后逐元素相加。

In [246]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

## 保存内存

[**运行操作可能会导致分配新的内存以存储结果。**]
例如，如果我们写 `Y = X + Y`，
我们取消了 `Y` 原来指向的张量的引用，
而是让 `Y` 指向新分配的内存。
我们可以用 Python 的 `id()` 函数来演示这个问题，
该函数可以给出内存中引用对象的确切地址。
请注意，在我们运行 `Y = Y + X` 之后，
`id(Y)` 指向了一个不同的位置。
这是因为 Python 首先计算 `Y + X`，
为结果分配新的内存，
然后让 `Y` 指向这个新的内存位置。

In [247]:
before = id(Y)
Y = Y + X
id(Y) == before

False

这可能有两个原因不理想。
首先，我们不想一直
无谓地分配内存。
在机器学习中，我们经常有
数百兆字节的参数
并且每秒更新多次。
只要有可能，我们希望*就地*执行这些更新。
其次，我们可能会从多个变量指向
相同的参数。
如果我们不就地更新，
我们必须小心更新所有这些引用，
以免出现内存泄漏
或无意中引用了过时的参数。

幸运的是，（执行就地操作）很简单。我们可以通过使用切片表示法 `Y[:] = <表达式>` 将操作的结果赋值给预先分配的数组 `Y`。为了说明这个概念，我们在初始化张量 `Z` 后覆盖其值，使用 `zeros_like` 使其与 `Y` 具有相同的形状。

In [248]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 140558549083504
id(Z): 140558549083504


[**如果`X`的值在后续计算中不再使用，我们也可以使用`X[:] = X + Y`或`X += Y`来减少操作的内存开销。**]

In [249]:
before = id(X)
X += Y
id(X) == before

True

## 转换为其他 Python 对象

[**转换为NumPy张量 (`ndarray`)**]，或者反过来，都是很容易的。
torch张量和NumPy数组
将共享它们底层的内存，
并且通过就地操作更改其中一个
也会更改另一个。

In [250]:
A = X.numpy()
B = torch.from_numpy(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

要（将大小为1的张量转换为Python标量），我们可以调用`item`函数或Python的内置函数。

In [251]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

## 摘要

张量类是深度学习库中存储和操作数据的主要接口。
张量提供了多种功能，包括构造例程；索引和切片；基本数学运算；广播；内存高效的赋值；以及与其他Python对象之间的转换。

## 练习

1. 运行本节中的代码。将条件语句 `X == Y` 更改为 `X < Y` 或 `X > Y`，然后看看能得到什么样的张量。
1. 将广播机制中按元素操作的两个张量替换为其他形状，例如3维张量。结果是否符合预期？

[讨论](https://discuss.d2l.ai/t/27)